# PriMAT: Multi-animal tracking

In this notebook, we want to demonstrate how to train a tracking model from a few hundred frames. We will train a model that is able to track lemurs and feeding boxes with labelled images which can be downloaded [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Training Images / LemurBoxSep22 ).

In [4]:
!nvidia-smi
!python3 --version

Wed Dec 10 23:32:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install yacs==0.1.8
!pip install opencv-python
!pip install progress==1.6
!pip install scikit-learn==1.2.2

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html


In [6]:
!git clone https://github.com/ecker-lab/PriMAT-tracking.git
%cd PriMAT-tracking/
!mkdir data
!mkdir models
!mkdir exp
!mkdir videos

Cloning into 'PriMAT-tracking'...
remote: Enumerating objects: 8971, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 8971 (delta 0), reused 0 (delta 0), pack-reused 8968 (from 2)
Receiving objects: 100% (8971/8971), 144.93 MiB | 33.79 MiB/s, done.
Resolving deltas: 100% (1789/1789), done.
Updating files: 100% (8972/8972), done.
/content/PriMAT-tracking


## Training

### Extract data

You have to upload the folder with the data to the colab space on the left. I uploaded it as a tar file and extracted it into the folder "data". The only important thing is that after this step you have training material in the folder data.

In [ ]:
%cd data
!tar -xvf ../../LemurBoxSep22.zip >/dev/null!tar
%cd ..

### Run training script

In the following cell you start the training. Before this you have to make sure a few things:
- The data_cfg file has to point to the correct location of your data. Open it (it is in PriMAT-tracking/src/lib/cfg/lemur-box.json) and adapt the root (for me it is /content/PriMAT-tracking/data/ in colab).
- Give your experiment a name (--exp_id) so that you can find the model afterwards in exp/mot/exp_id/.

In [ ]:
%%shell
cd src
python train.py mot --exp_id colab_test\
                    --load_model ''\
                    --num_epochs 10\
                    --lr_step 5\
                    --lr '1e-4'\
                    --data_cfg '../src/lib/cfg/lemur_box.json'\
                    --store_opt\
                    --arch hrnet_32\
                    --gpus 0\
                    --batch_size 2\
                    --seed 13\
                    --reid_cls_names lemur,box\
                    --val_intervals 10\
                    --save_all
cd ..

Your models will be saved in exp/mot/exp_id and end with .pth.

## Inference

### Apply to videos

If you want to have video output, you will need to activate ffmpeg.

In [ ]:
!apt-get update && apt-get install -y ffmpeg

- If you want the model you just trained, you can directly change the path to ../exp/mot/exp_id/model_last.pth (or any other model you want). Alternatively, we can use the pretrained lemur model from [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Models).
- You can upload your own videos or a validation video (e.g. Eval8.mp4) into data/Videos. Videos can be downloaded [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> ValidationVideos/lemur_videos_eval/Videos/).
- If you set output_format to video, a video will be saved to output_root/output_name. If you set it to text, only the tracking output as a .txt file will be saved.

In [ ]:
!pip install motmetrics==1.2.0
!pip install lap==0.4.0
!pip install cython_bbox==0.1.3

In [ ]:
%%shell

cd src

python demo.py mot  --load_tracking_model ../models/lemur_tracking.pth\
                    --conf_thres 0.02\
                    --det_thres 0.5\
                    --new_overlap_thres 0.8\
                    --sim_thres 0.8\
                    --input_video ../data/Videos/Eval8.mp4\
                    --output_root ../videos/test/\
                    --output_name test_video\
                    --store_opt\
                    --line_thickness 2\
                    --debug_info\
                    --arch hrnet_32\
                    --output_format video\
                    --reid_cls_names "lemur,box"\
                    --proportion_iou 0.2\
                    --double_kalman


cd ..